In [1]:
import pandas as pd
from datetime import datetime, timedelta
from collections import OrderedDict
import numpy as np

In [2]:
df = pd.read_excel('/Users/j-yeom/workspace/asakai/charge/charge-total.xlsx', sheet_name = ['03-03','2018-04','2018-05','2018-06'])
#print(df)
df1 = pd.DataFrame()
df1['date'] = (df['03-03']['date']).append(df['2018-04']['date']).append(df['2018-05']['date']).append(df['2018-06']['date'])
df1['date'] = pd.to_datetime(df1['date'], format='%Y/%m/%d %H:%M')
df1['date_month'] = df1['date'].dt.strftime('%Y-%m')
df1['userId'] = (df['03-03']['userId']).append(df['2018-04']['userId']).append(df['2018-05']['userId']).append(df['2018-06']['userId'])
#df1['userId'] = df['03-03']['userId']
df1['userId'] = df1.userId.apply(str)
#print(df1)

In [3]:
monthList = df1.date_month.unique()
print(monthList)
userList = []
for month in monthList:
    if(len(userList) == 0):
        userList.append(list(df1.userId.where(df1['date_month'] == month).unique()))
    else:
        toAppend = set(df1.userId.where(df1['date_month'] == month).unique())
        # remove all the previous months' users
        toRemove = set().union(*userList)
        userList.append(list(toAppend - toRemove))
#print(userList) # userList will now contain lists of first users from each month.
userObj = {}
count = 0
firstUserCount = []
for month in monthList:
    userObj[month] = userList[count]
    firstUserCount.append(len(userObj[month]))
    count+=1

print(firstUserCount)

['2017-03' '2017-04' '2017-05' '2017-06' '2017-07' '2017-08' '2017-09'
 '2017-10' '2017-11' '2017-12' '2018-01' '2018-02' '2018-03' '2018-04'
 '2018-05' '2018-06']
[769, 507, 771, 415, 390, 790, 509, 289, 242, 691, 719, 297, 397, 411, 388, 510]


In [4]:
userList = {}
for month in monthList:
    userList[month] = df1.userId.where(df1['date_month'] == month).unique()  # << unique user of each month
#print(userList) 각 월의 유저아이디를 담고 있는 객체

def match_all(rangeArr, subRangeArr):
    arr = {}
    for month in rangeArr:
        arr[month] = []
        for sub_month in subRangeArr:  
            # figure out if each month unique user existed in the first charger user group
            arr[month].append(np.isin(userList[sub_month],userObj[month]).sum())
    return arr

dates = ["2017-03-01", "2018-06-30"]
start, end = [datetime.strptime(_, "%Y-%m-%d") for _ in dates]
result = list(OrderedDict(((start + timedelta(_)).strftime(r"%Y-%m"), None) for _ in range((end - start).days)).keys())
#print(for_march(result))
real_result = match_all(result, result) # use same time range for both compare-ref and compare-ee
#print(real_result)

In [9]:
df2 = df1.groupby('date_month')[['userId']].nunique()
#df2['userId'] = firstUserCount
for month, month_list in real_result.items():
    df2[month] = month_list
df2 = df2.transpose()

df2 = df2.replace(0, np.nan)
print(df2)
writer = pd.ExcelWriter('pollet_charge1.xlsx', engine='xlsxwriter')
df2.to_excel(writer, sheet_name='Sheet1')
writer.save()
#df2.to_json('pollet_charge_split.json', orient='split')

            userId  2017-03  2017-04  2017-05  2017-06  2017-07  2017-08  \
date_month                                                                 
2017-03        768      768      NaN      NaN      NaN      NaN      NaN   
2017-04        757      250    507.0      NaN      NaN      NaN      NaN   
2017-05       1165      235    159.0    771.0      NaN      NaN      NaN   
2017-06       1080      252    167.0    246.0    415.0      NaN      NaN   
2017-07       1029      221    124.0    175.0    119.0    390.0      NaN   
2017-08       1674      249    142.0    196.0    134.0    163.0    790.0   
2017-09       1559      254    145.0    197.0    105.0    146.0    203.0   
2017-10       1170      184    116.0    160.0     75.0    110.0    125.0   
2017-11       1313      212    114.0    188.0     74.0    111.0    125.0   
2017-12       2157      270    165.0    259.0    132.0    126.0    153.0   
2018-01       1573      114     73.0    121.0     49.0     83.0     75.0   
2018-02     